In [1]:
from __future__ import print_function
import tensorflow as tf
import numpy as np
import random
import os
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
with open("./data/friends/all_scripts.txt") as scripts_fileobj:
    all_scripts = scripts_fileobj.read().strip().lower().decode('utf8').encode('ascii', errors='ignore')

In [3]:
def preprocess_text(text):
    lines = text.split("\n")
    dialogues = []
    dialogue = []
    for line in lines:
        if ":" in line:
            dialogues.append(" ".join(dialogue))
            dialogue = []
            dialogue.append(line)
        else:
            dialogue.append(line)
    
    text = "\n".join(dialogues)
    punctuations = set(re.findall(r"[^a-zA-Z0-9 ]",text))
    for punctuation in punctuations:
        if punctuation == "\n":
            text = text.replace(punctuation," NEWLINE ")
        else:
            text = text.replace(punctuation," "+punctuation+" ")
            
        
    return text

In [4]:
def remove_infrequent_tokens(tokens,min_count=10):
    word_count = {}
    new_tokens = []
    vocab = []
    for token in tokens:
        if token in word_count:
            word_count[token] +=1
        else:
            word_count[token]=1
    for token_word in tokens:
        if word_count[token_word]>min_count:
            new_tokens.append(token_word)
    return new_tokens

In [5]:
all_scripts_cleaned = preprocess_text(all_scripts)
tokens = all_scripts_cleaned.split()


In [6]:
# print(len(tokens))
cleaned_tokens = remove_infrequent_tokens(tokens)
print("Number of cleaned tokens: {}".format(len(cleaned_tokens)))

Number of cleaned tokens: 1300961


In [7]:
vocab = list(set(cleaned_tokens))
print("Vocal length: {}".format(len(vocab)))

Vocal length: 3567


In [8]:
vocab_length = len(vocab)
characters2id = dict((c, i) for i, c in enumerate(vocab))
id2characters = dict((i, c) for i, c in enumerate(vocab))
section_length = 20
step = 2
sections = []
section_labels = []
for i in range(0,len(cleaned_tokens)-section_length,step):
    section_in = cleaned_tokens[i:i+section_length]
    section_out = cleaned_tokens[i+section_length]
    sections.append([characters2id[word] for word in section_in])
    section_labels.append(characters2id[section_out])

print("Number of training examples: {}".format(len(sections)))

Number of training examples: 650471


In [9]:
X = np.reshape(sections, (len(sections), section_length, 1))
print(X.shape)
X = X / float(vocab_length)
y = np.zeros((len(sections),vocab_length))
for i,section in enumerate(sections):
    y[i,section_labels[i]] = 1

(650471, 20, 1)


In [10]:
print(X.shape,y.shape)

(650471, 20, 1) (650471, 3567)


In [11]:
model = Sequential()
model.add(LSTM(1024, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
filepath="./model/model3_friends/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=60, batch_size=128, callbacks=callbacks_list,initial_epoch=20)

Epoch 21/60
650471/650471 [==============================] - 542s 834us/step - loss: 5.2049
Epoch 22/60
650471/650471 [==============================] - 529s 814us/step - loss: 4.8551
Epoch 23/60
650471/650471 [==============================] - 531s 816us/step - loss: 4.7099
Epoch 24/60
650471/650471 [==============================] - 531s 816us/step - loss: 4.5989
Epoch 25/60
650471/650471 [==============================] - 530s 815us/step - loss: 4.4947
Epoch 26/60
650471/650471 [==============================] - 530s 814us/step - loss: 4.3859
Epoch 27/60
650471/650471 [==============================] - 534s 820us/step - loss: 4.2696
Epoch 28/60
650471/650471 [==============================] - 534s 821us/step - loss: 4.1462
Epoch 29/60
650471/650471 [==============================] - 535s 823us/step - loss: 4.0136
Epoch 30/60
650471/650471 [==============================] - 535s 823us/step - loss: 3.8782
Epoch 31/60
650471/650471 [==============================] - 537s 825us/step - l

650471/650471 [==============================] - 530s 814us/step - loss: 2.0642
Epoch 50/60
650471/650471 [==============================] - 528s 812us/step - loss: 2.0128
Epoch 51/60
650471/650471 [==============================] - 528s 812us/step - loss: 1.9601
Epoch 52/60
650471/650471 [==============================] - 528s 812us/step - loss: 1.9124
Epoch 53/60
650471/650471 [==============================] - 529s 813us/step - loss: 1.8661
Epoch 54/60
650471/650471 [==============================] - 528s 812us/step - loss: 1.8185
Epoch 55/60
650471/650471 [==============================] - 529s 813us/step - loss: 1.7745
Epoch 56/60
650471/650471 [==============================] - 529s 813us/step - loss: 1.7371
Epoch 57/60
650471/650471 [==============================] - 529s 813us/step - loss: 1.6984
Epoch 58/60
650471/650471 [==============================] - 528s 812us/step - loss: 1.6619
Epoch 59/60
650471/650471 [==============================] - 528s 811us/step - loss: 1.6281


In [14]:
# Testing
test_filename = "./model/model3_friends/weights-improvement-60-1.5949.hdf5"
model.load_weights(test_filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

start = np.random.randint(0, len(sections)-1)
pattern = sections[start]
print("Seed:", " ".join([id2characters[idx] for idx in pattern]).replace("NEWLINE","\n"))
predictions = []
# generate characters
for i in range(500):
#     print([id2characters[idx] for idx in pattern])    
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_length)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = id2characters[index]
    seq_in = [id2characters[value] for value in pattern]
    predictions.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print(" ".join(predictions).replace("NEWLINE","\n"))

Seed: will be fun ! ohh ! yay ! oh ! okay , ooh , lets plan the wedding reception .

 rachel : oh , i , i wouldn think . s up for , . 
 monica : yeah , he think have you what they of 
 party 
 me : no , you have re right . i . don i hate i 
 you . i , who would everyone , you , i i up gotta way and . 
 mr : you , . 
 phoebe : you , but was , . 
 monica : ( , yeah . . ' ( t the her her her she 
 , : t like 
 chandler : alright , no . 
 ross : oh , - . 
 joey : oh , bye . 
 ross : hey , no , ' , ok , be , ross , don ' t be why 
 rachel at hands . 
 gavin : i , i ' m ' . we ' 
 look . and on bathroom is they i . . 
 phoebe : ( , you ) ) ) 
 ! 
 ( , start the over joey the . . ) tries , : oh , this , you really that up like 
 rachel : yeah , i , i , i know ll . i removes as so ' that ive mine i , i have to i , and that with would now . 
 rachel : oh ? 
 ross : no , you , ' you little to your ? 
 rachel : oh , i ' m sorry . . . 
 ross : y i . , ' . 
 joey : ok , i . . . ' , i . . . she ' m 

In [ ]:
print(vocab)